# Incorporate results of manual accuracy evaluation to select fixations
Status: Implemented
 -- Fully automatized

In [1]:
from tqdm.notebook import tqdm
from utils.utils import *

In [2]:
# get all files from participants
folders = get_participant_folder_per_participant()
sequence_orders = get_sequence_order_per_participant()
assert (set(folders.keys()) == set(sequence_orders.keys()))
snippets = {p: sequence_orders[p][SNIPPET].to_list()
            for p in sequence_orders}

In [3]:
global FIXATION_CORRECTION_ITERATION 
FIXATION_CORRECTION_ITERATION[0] = 4
print('Fixation correction iteration:', current_fixation_correction_iteration())
print('Previous Fixation correction iteration:', previous_fixation_correction_iteration())
x_offsets = get_x_offsets()
x_offsets

Fixation correction iteration: 4
Previous Fixation correction iteration: 3


{'003': {},
 '004': {},
 '005': {},
 '007': {},
 '008': {},
 '009': {},
 '010': {},
 '011': {},
 '012': {'v0': 100, 'v1': 50, 'v2': 80},
 '013': {'v0': 20},
 '014': {},
 '015': {},
 '016': {},
 '017': {},
 '018': {},
 '019': {},
 '020': {},
 '021': {},
 '022': {},
 '023': {},
 '024': {},
 '101': {},
 '102': {},
 '106': {}}

### Include manual evaluation
* check manual accuracy evaluation file
* exclude unusable trials
* copy selected fixations

In [4]:
all_usable = True
participant_exclusions = {}
for participant in tqdm(folders):

    print('----------------------------------------------')
    print(PARTICIPANT, participant)
    manual_accuracy_evaluation_path = get_manual_accuracy_evaluation_path(
        participant, performed=True)
    if not manual_accuracy_evaluation_path.exists():
        print('Performed evaluation path not found:',
              manual_accuracy_evaluation_path.as_posix())
        continue
    print('(01/03) Check manual evaluation file')
    manual_evaluation = pd.read_csv(
        manual_accuracy_evaluation_path, sep=SEPARATOR, index_col=False, dtype={PARTICIPANT: str})
    usable = check_manual_accuracy_evaluation(
        manual_evaluation, participant, snippets[participant])
    if not usable:
        print('\n--The fixations for this participant have to be re-evaluated in another iteration.')
        continue

    print('(02/03) Exclude trials without suitable corrections (excluded)')
    to_be_excluded = {row[SNIPPET]: row[MANUAL_ACCURACY_EVALUATION_REASON] for _, row in manual_evaluation.iterrows(
    ) if row[MANUAL_ACCURACY_EVALUATION_CHOICE] == FIXATION_CORRECTION_ALGORITHM_SNIPPET_EXCLUDED}
    participant_exclusions[participant] = to_be_excluded
    print('(03/03) Copy chosen fixations')
    for _, row in manual_evaluation.iterrows():
        algo = row[MANUAL_ACCURACY_EVALUATION_CHOICE]
        final_iteration = row[MANUAL_ACCURACY_EVALUATION_ITERATION]
        if algo == FIXATION_CORRECTION_ALGORITHM_SNIPPET_EXCLUDED:
            continue
        snippet_fixations_path = get_fixations_path(participant, row[SNIPPET], fixation_cross=False, modified_version=algo, no_outliers_if_exists=(
            algo == FIXATION_CORRECTION_ALGORITHM_ORIGINAL), iteration=lambda:final_iteration)
        fixation_data = pd.read_csv(
            snippet_fixations_path, index_col=False, sep=SEPARATOR, dtype={PARTICIPANT: str})
        if fixation_data.shape[0]<MIN_FIXATION_COUNT_TRIAL:
            print(f'{row[SNIPPET]}: trial with less than {MIN_FIXATION_COUNT_TRIAL} fixations, thus excluded')
            to_be_excluded[row[SNIPPET]]='Too few fixations'
            continue
        if participant in x_offsets:
            x_offset = get_snippet_x_offset(x_offsets[participant], row[SNIPPET])
            if x_offset:
                fixation_data[FIXATION_COLUMN_X] = fixation_data[FIXATION_COLUMN_X]+x_offset
        fixation_data.to_csv(get_selected_fixations_path(
            participant, row[SNIPPET]), sep=SEPARATOR, index=False)
    if to_be_excluded:
        pass
        update_exclusions(participant, SNIPPET, VISUAL,
                          {snippet: f'01d7_PRE_VIS_Gaze_Selection_Fixation-it{current_fixation_correction_iteration()}: No fixation correction selected due to unsuitable accuracy. {reason}' for snippet, reason in to_be_excluded.items()})


  0%|          | 0/24 [00:00<?, ?it/s]

----------------------------------------------
Participant 003
(01/03) Check manual evaluation file
(02/03) Exclude trials without suitable corrections (excluded)
(03/03) Copy chosen fixations
----------------------------------------------
Participant 004
(01/03) Check manual evaluation file
(02/03) Exclude trials without suitable corrections (excluded)
(03/03) Copy chosen fixations
----------------------------------------------
Participant 005
(01/03) Check manual evaluation file
(02/03) Exclude trials without suitable corrections (excluded)
(03/03) Copy chosen fixations
----------------------------------------------
Participant 007
(01/03) Check manual evaluation file
(02/03) Exclude trials without suitable corrections (excluded)
(03/03) Copy chosen fixations
----------------------------------------------
Participant 008
(01/03) Check manual evaluation file
(02/03) Exclude trials without suitable corrections (excluded)
(03/03) Copy chosen fixations
-----------------------------------

In [5]:
participant_exclusion_count = {participant:len(participant_exclusions[participant]) for participant in participant_exclusions}
print(f'exclusion no by participant { participant_exclusion_count}')
print('total exclusion no', sum(participant_exclusion_count.values()))
print('average exclusion by participant', round(sum(participant_exclusion_count.values())/len(participant_exclusion_count),2),)
print('exclusion rate', round(100*sum(participant_exclusion_count.values())/(72*len(participant_exclusion_count)),2),'%')

exclusion no by participant {'003': 19, '004': 28, '005': 11, '007': 7, '008': 16, '009': 4, '010': 12, '011': 6, '012': 6, '013': 5, '014': 3, '015': 1, '016': 12, '017': 14, '018': 7, '019': 12, '020': 6, '021': 1, '022': 12, '023': 3, '024': 2, '101': 6, '102': 4, '106': 30}
total exclusion no 227
average exclusion by participant 9.46
exclusion rate 13.14 %


In [6]:
if all_usable:
    print('All fixations were corrected without concerns, you can continue with the pipeline.')
else:
    raise Exception(f'The fixations for at least one participant have to be re-evaluated in another iteration. See above.')

All fixations were corrected without concerns, you can continue with the pipeline.
